In [1]:
import pandas as pd
import numpy as np
import glob
import math

In [2]:
path = './Data/extraction/'
files = glob.glob(path + "*.csv")
for file in files:
    name = file.replace('.csv','')
    name = name.replace('_20221221','')
    name = name.replace('./Data/extraction/','')        
    vars()['df_'+str(name)] = pd.read_csv(file).drop('Unnamed: 0', axis=1, errors='ignore')
    print('df_'+str(name))

df_itemid_and_periods
df_mycloud
df_scenarios
df_item_info
df_item_trend
df_tmp_hosts_zabbix
df_cockpit
df_item_ts_seasonalized


In [ ]:
#df_cockpit.head(50)
#df_item_info.head(50)
#df_item_trend.head(50)
#df_mycloud.head(50)
#df_tmp_hosts_zabbix.head(50)
#df_itemid_and_periods.head(50)
#df_item_ts_seasonalized.head()

## DATAFRAMES DEFINITION

In [3]:
def df_global_creation(df_itemid_and_periods=df_itemid_and_periods, df_cockpit=df_cockpit, df_tmp_hosts_zabbix=df_tmp_hosts_zabbix, df_item_info=df_item_info, df_item_trend=df_item_trend):
    
    df_itemid_and_periods.loc[df_itemid_and_periods.period1>0,'season']=1
    df_itemid_and_periods['season'] = df_itemid_and_periods['season'].fillna(0)
    df_itemid_and_periods = df_itemid_and_periods.drop(['period2', 'period3'], axis=1, errors='ignore')

    df_appli_server = pd.DataFrame(df_cockpit['name_server'])
    df_appli_server['ram'] = df_cockpit['ram']
    df_appli_server['number_core'] = df_cockpit['number_core']
    df_appli_server['number_cpu'] = df_cockpit['number_cpu']
    df_appli_server['type_cpu'] = df_cockpit['type_cpu']

    df_appli_server = df_appli_server.merge(df_tmp_hosts_zabbix, left_on ="name_server", right_on="host")
    df_appli_server = df_appli_server.merge(df_item_info, on ="hostid")
    df_appli_server = df_appli_server.drop(['item_type'], axis=1)
    df_item_trend_type = df_item_trend.drop(['clock', 'value_min','value_avg','value_max'], axis=1)
    df_item_trend_type.drop_duplicates(keep = 'first', inplace=True)
    df_appli_server = pd.merge(df_appli_server, df_item_trend_type, on = ['itemid'])
    df_appli_server = df_appli_server.drop_duplicates()
    df_appli_server = df_appli_server[df_appli_server.item_type != "citrix"]
    df_appli_server = df_appli_server.merge(df_itemid_and_periods, on ="itemid")

    return df_appli_server

df_appli_server = df_global_creation()

In [4]:
def df_local_creation(df_item_ts_seasonalized=df_item_ts_seasonalized):
    item_ids = df_item_ts_seasonalized["itemid"].unique()
    df_item_ts_30 = pd.DataFrame(columns = df_item_ts_seasonalized.columns)
    for item_id in item_ids:
        sample = df_item_ts_seasonalized[df_item_ts_seasonalized.itemid==item_id].sort_values('ds', ascending=True).tail(30)
        df_item_ts_30  = pd.concat([df_item_ts_30, sample])
    df_item_ts_30["item_type"] = df_item_ts_30["item_type"].replace('mem_pused','mem')
    return df_item_ts_30
df_item_ts_30 = df_local_creation()

## SCENARIOS PREPROCESSING

In [5]:
def count_successive_above_or_under_threshold(df_item_trend, metric="value_avg", above=True, threshold=90):
    df = df_item_trend.copy()
    target_col = "sdays_above_threshold"
    df[target_col] = 0  # initialize target column
    iter = 0
    for itemid in df["itemid"].unique():
        thisItem_df = df.loc[df["itemid"]==itemid]
        idxValues = thisItem_df.index
        s = thisItem_df[metric]
    
        # 1 when condition satisfied for just one row, else 0
        s2 = (threshold <= s)*1 if above else (s <= threshold)*1
        # n when 1 at this row and n-1 row above
        s3 = s2 * (s2.groupby((s2 != s2.shift()).cumsum()).cumcount() + 1)
        # insert values in target column
        df.loc[idxValues, [target_col]] = s3
    return df

In [6]:
def usage(df_item_trend=df_item_trend,df_appli_server=df_appli_server, metric="value_avg"):
    df_sdays = count_successive_above_or_under_threshold(df_item_trend, metric=metric, above=True, threshold=99.9)
    df_sdays = df_sdays.drop(['value_min','value_avg','value_max','item_type'], axis=1)
    df_item_trend_2 = pd.merge(df_item_trend, df_sdays, on = ["itemid","clock"])
    metric_sdays = metric + '_sdays'
    df_item_trend_2[metric_sdays] = df_item_trend_2[metric] * df_item_trend_2["sdays_above_threshold"].replace(0, 1)
    df_temp = pd.DataFrame(df_item_trend_2["itemid"])
    df_temp[metric_sdays] = df_item_trend_2[metric_sdays]
    df_temp = df_temp.groupby(df_temp["itemid"]).mean()
    df_temp = df_temp.reset_index()

    df_temp2 = pd.DataFrame(df_item_trend_2["itemid"])
    df_temp2["sdays_above_threshold"] = df_item_trend_2["sdays_above_threshold"]
    df_temp2 = df_temp2.groupby(df_temp2["itemid"]).max()
    df_temp2 = df_temp2.reset_index()

    df_temp = df_temp.merge(df_temp2, on=["itemid"])
    
    df_appli_usage = pd.merge(df_appli_server, df_temp, on = ['itemid'], suffixes=('', '_y'))
    df_appli_usage.drop(df_appli_usage.filter(regex='_y$').columns, axis=1, inplace=True)
    return df_appli_usage

df_appli_usage = usage()


## SCENARIOS FUNCTIONS

In [7]:
def config_optim(df=df_appli_usage, threshold_min_cpu=60, threshold_max_cpu=80, threshold_min_ram=70, threshold_max_ram=80):
    
    cpu_config = [1,2,4,8,12,16]
    ram_config = [2,4,6,8,12,16,24,32,64,128]
    lnew_cpu=[]
    lnew_ram=[]

    for i in range(len(df)):
        
        season = df.iloc[i]["season"]
        if season == 0:
            item_type = df.iloc[i]["item_type"]
            usage = df.iloc[i]["value_avg_sdays"]
            cpu = df.iloc[i]["number_cpu"]
            ram_format = df.iloc[i]["ram"]
            ram = min(ram_config, key=lambda x:abs(x-ram_format/1000))

            if item_type == 'cpu':
                lnew_ram.append(ram)
                if usage != 0:
                    if usage < threshold_min_cpu:
                        ratio = round(threshold_min_cpu/usage)
                        new_cpu = min(cpu_config, key=lambda x:abs(x-cpu/ratio))
                        lnew_cpu.append(new_cpu)
                    elif usage > threshold_max_cpu:
                        ratio = math.ceil(usage/threshold_max_cpu)
                        new_cpu = min(cpu_config, key=lambda x:abs(x-cpu*ratio))
                        lnew_cpu.append(new_cpu)
                    else:
                        lnew_cpu.append(cpu)
                else:
                    lnew_cpu.append(cpu)

            elif item_type == 'mem':
                lnew_cpu.append(cpu)
                if usage != 0:
                    if usage < threshold_min_ram:
                        ratio = round(threshold_min_ram/usage)
                        new_ram = min(ram_config, key=lambda x:abs(x-ram/ratio))
                        lnew_ram.append(new_ram)
                    elif usage > threshold_max_ram:
                        ratio = math.ceil(usage/threshold_max_ram)
                        new_ram = min(ram_config, key=lambda x:abs(x-ram*ratio))
                        lnew_ram.append(new_ram)
                    else:
                        lnew_ram.append(ram)
                else:
                    lnew_ram.append(ram)
            else:
                lnew_ram.append(ram)
                lnew_cpu.append(cpu)
        else:
            lnew_ram.append(ram)
            lnew_cpu.append(cpu)   

    df["new_cpu_optim"] = lnew_cpu
    df["new_ram_optim"] = lnew_ram

    count_cpu_sup = 0
    count_cpu = 0
    count_cpu_inf = 0
    count_ram_sup = 0
    count_ram = 0
    count_ram_inf = 0


    for i in range(len(df)):
        season = df.iloc[i]["season"]
        if season == 0:
            cpu = df.iloc[i]["number_cpu"]
            ram = min(ram_config, key=lambda x:abs(x-df.iloc[i]["ram"]/1000))
            new_cpu = df.iloc[i]["new_cpu_optim"]
            new_ram = df.iloc[i]["new_ram_optim"]
            item_type = df.iloc[i]["item_type"]
            if item_type == "cpu":
                if new_cpu > cpu:
                    count_cpu_sup += 1
                elif new_cpu == cpu:
                    count_cpu += 1
                else:
                    count_cpu_inf += 1
            elif item_type == "mem":
                if new_ram > ram:
                    count_ram_sup += 1
                elif new_ram == ram:
                    count_ram += 1
                else:
                    count_ram_inf += 1

    print( count_cpu_sup, count_cpu, count_cpu_inf, count_ram_sup, count_ram, count_ram_inf )
    return df
                
df_appli_usage = config_optim()
df_appli_usage.head()

1 25 1949 116 1104 869


,name_server,ram,number_core,number_cpu,type_cpu,hostid,host,itemid,item_type,period1,season,value_avg_sdays,sdays_above_threshold,new_cpu_optim,new_ram_optim
0,SWDCFRAV9779,16777,2,2,INTEL(R) XEON(R) SILVER 4314 CPU @ 2.40GHZ,100100000175011,SWDCFRAV9779,100100034653281,cpu,NaN,0.0,5.224110,0,1,16
1,SWDCFRAV9779,16777,2,2,INTEL(R) XEON(R) SILVER 4314 CPU @ 2.40GHZ,100100000175011,SWDCFRAV9779,100100034653289,mem,NaN,0.0,28.252698,0,2,8
2,SWPAFRETR078,16777,8,8,INTEL(R) XEON(R) CPU E5-2620 V4 @ 2.10GHZ,100100000225754,SWPAFRETR078,100100043158924,cpu,NaN,0.0,5.158280,0,1,16
3,SWPAFRETR078,16777,8,8,INTEL(R) XEON(R) CPU E5-2620 V4 @ 2.10GHZ,100100000225754,SWPAFRETR078,100100043158932,mem,NaN,0.0,24.242146,0,8,6
4,SWPAFRETR079,16777,8,8,INTEL(R) XEON(R) CPU E5-2620 V4 @ 2.10GHZ,100100000225753,SWPAFRETR079,100100043138385,cpu,NaN,0.0,8.268402,0,1,16


In [9]:
def config_constraint_prio(df=df_appli_usage, threshold_min_cpu=60, threshold_max_cpu=80, threshold_min_ram=70, threshold_max_ram=80, max_cpu=35, max_ram=0):    
    
    cpu_config = [1,2,4,8,16]
    ram_config = [2,4,6,8,12,16,24,32,64,128]
    lnew_cpu=[]
    lnew_ram=[]

    for i in range(len(df)):
        item_type = df.iloc[i]["item_type"]
        usage = df.iloc[i]["value_avg_sdays"]
        cpu = df.iloc[i]["number_cpu"]
        ram_format = df.iloc[i]["ram"]
        ram = min(ram_config, key=lambda x:abs(x-ram_format/1000))

        if item_type == 'cpu':
            lnew_ram.append(ram)
            if usage != 0:
                if usage < threshold_min_cpu:
                    ratio = round(threshold_min_cpu/usage)
                    new_cpu = min(cpu_config, key=lambda x:abs(x-cpu/ratio))
                    max_cpu = max_cpu + cpu - new_cpu
                    lnew_cpu.append(new_cpu)
                else:
                    lnew_cpu.append(cpu)
            else:
                lnew_cpu.append(cpu)

        elif item_type == 'mem':
            lnew_cpu.append(cpu)
            if usage != 0:
                if usage < threshold_min_ram:
                    ratio = round(threshold_min_ram/usage)
                    new_ram = min(ram_config, key=lambda x:abs(x-ram/ratio))
                    max_ram = max_ram + ram - new_ram
                    lnew_ram.append(new_ram)
                else:
                    lnew_ram.append(ram)
            else:
                lnew_ram.append(ram)
        else:
            lnew_cpu.append(cpu)
            lnew_ram.append(ram)

    df["new_cpu_constraint"] = lnew_cpu
    df["new_ram_constraint"] = lnew_ram

    df = df.sort_values("value_avg_sdays" , ascending=False)
    for i in range(len(df)):

        item_type = df.iloc[i]["item_type"]
        usage = df.iloc[i]["value_avg_sdays"]
        cpu = df.iloc[i]["number_cpu"]
        ram_format = df.iloc[i]["ram"]
        ram = min(ram_config, key=lambda x:abs(x-ram_format/1000))

        if item_type == 'cpu':
            if usage > threshold_max_cpu:
                ratio = math.ceil(usage/threshold_max_cpu)
                new_cpu = min(cpu_config, key=lambda x:abs(x-cpu*ratio))
                if new_cpu <= max_cpu:
                    df.at[i, 'new_cpu'] = new_cpu
                    max_cpu = max_cpu + cpu - new_cpu

        elif item_type == 'mem':
            if usage > threshold_max_ram:
                ratio = math.ceil(usage/threshold_max_ram)
                new_ram = min(ram_config, key=lambda x:abs(x-ram*ratio))
                if new_ram <= max_ram:
                    df.at[i, 'new_ram'] = new_ram
                    max_ram = max_ram + ram - new_ram
        
    count_cpu_sup = 0
    count_cpu = 0
    count_cpu_inf = 0
    count_ram_sup = 0
    count_ram = 0
    count_ram_inf = 0


    for i in range(len(df)):
        season = df.iloc[i]["season"]
        if season == 0:
            cpu = df.iloc[i]["number_cpu"]
            ram = min(ram_config, key=lambda x:abs(x-df.iloc[i]["ram"]/1000))
            new_cpu = df.iloc[i]["new_cpu_constraint"]
            new_ram = df.iloc[i]["new_ram_constraint"]
            item_type = df.iloc[i]["item_type"]
            if item_type == "cpu":
                if new_cpu > cpu:
                    count_cpu_sup += 1
                elif new_cpu == cpu:
                    count_cpu += 1
                else:
                    count_cpu_inf += 1
            elif item_type == "mem":
                if new_ram > ram:
                    count_ram_sup += 1
                elif new_ram == ram:
                    count_ram += 1
                else:
                    count_ram_inf += 1

    print( count_cpu_sup, count_cpu, count_cpu_inf, count_ram_sup, count_ram, count_ram_inf )
    print( "remaining number of cpu: " + str(max_cpu) + ", remaining number of ram: " + str(max_ram) )
    return df_appli_usage
df_appli_usage = config_constraint_prio()
df_appli_usage.head()

1 25 1949 116 1104 869
remaining number of cpu: 5410, remaining number of ram: 7846


,name_server,ram,number_core,number_cpu,type_cpu,hostid,host,itemid,item_type,period1,season,value_avg_sdays,sdays_above_threshold,new_cpu_optim,new_ram_optim,new_cpu_constraint,new_ram_constraint
0,SWDCFRAV9779,16777,2,2,INTEL(R) XEON(R) SILVER 4314 CPU @ 2.40GHZ,100100000175011,SWDCFRAV9779,100100034653281,cpu,NaN,0.0,5.224110,0,1,16,1,16
1,SWDCFRAV9779,16777,2,2,INTEL(R) XEON(R) SILVER 4314 CPU @ 2.40GHZ,100100000175011,SWDCFRAV9779,100100034653289,mem,NaN,0.0,28.252698,0,2,8,2,8
2,SWPAFRETR078,16777,8,8,INTEL(R) XEON(R) CPU E5-2620 V4 @ 2.10GHZ,100100000225754,SWPAFRETR078,100100043158924,cpu,NaN,0.0,5.158280,0,1,16,1,16
3,SWPAFRETR078,16777,8,8,INTEL(R) XEON(R) CPU E5-2620 V4 @ 2.10GHZ,100100000225754,SWPAFRETR078,100100043158932,mem,NaN,0.0,24.242146,0,8,6,8,6
4,SWPAFRETR079,16777,8,8,INTEL(R) XEON(R) CPU E5-2620 V4 @ 2.10GHZ,100100000225753,SWPAFRETR079,100100043138385,cpu,NaN,0.0,8.268402,0,1,16,1,16


In [20]:
def config_optim_season(df=df_item_ts_30, threshold_min_cpu=60, threshold_max_cpu=80, threshold_min_ram=70, threshold_max_ram=80):
    
    cpu_config = [1,2,4,8,12,16]
    ram_config = [2,4,6,8,12,16,24,32,64,128]
    lnew_cpu=[]
    lnew_ram=[]

    for i in range(len(df)):
        item_type = df.iloc[i]["item_type"]
        usage_max = df.iloc[i]["yhat_upper"]
        usage_min = df.iloc[i]["yhat_lower"]
        cpu = df.iloc[i]["number_cpu"]
        ram = df.iloc[i]["ram_server"]

        if item_type == 'cpu':
            lnew_ram.append(ram)
            if usage_max != 0:
                if usage_max > threshold_max_cpu and usage_min < threshold_min_cpu:
                    ratio = math.ceil(usage_max/threshold_max_cpu)
                    new_cpu = min(cpu_config, key=lambda x:abs(x-cpu*ratio))
                    lnew_cpu.append(new_cpu)
                elif usage_max > threshold_max_cpu:
                    ratio = math.ceil(usage_max/threshold_max_cpu)
                    new_cpu = min(cpu_config, key=lambda x:abs(x-cpu*ratio))
                    lnew_cpu.append(new_cpu)
                elif usage_min < threshold_min_cpu:
                    ratio = round(threshold_min_cpu/usage_min)
                    new_cpu = min(cpu_config, key=lambda x:abs(x-cpu/ratio))
                    lnew_cpu.append(new_cpu)
                else:
                    lnew_cpu.append(cpu)
            else:
                lnew_cpu.append(cpu)

        elif item_type == 'mem':
            lnew_cpu.append(cpu)
            if usage_max != 0:
                if usage_min < threshold_min_ram and usage_max > threshold_max_ram:
                    ratio = math.ceil(usage_max/threshold_max_ram)
                    new_ram = min(ram_config, key=lambda x:abs(x-ram*ratio))
                    lnew_ram.append(new_ram)
                elif usage_max > threshold_max_ram:
                    ratio = math.ceil(usage_max/threshold_max_ram)
                    new_ram = min(ram_config, key=lambda x:abs(x-ram*ratio))
                    lnew_ram.append(new_ram)
                elif usage_min < threshold_min_ram:
                    ratio = round(threshold_min_ram/usage_min)
                    new_ram = min(ram_config, key=lambda x:abs(x-ram/ratio))
                    lnew_ram.append(new_ram)
                else:
                    lnew_ram.append(ram)
            else:
                lnew_ram.append(ram)
        else:
            lnew_cpu.append(cpu)
            lnew_ram.append(ram)

    df["new_cpu_optim"] = lnew_cpu
    df["new_ram_optim"] = lnew_ram

    count_cpu_sup = 0
    count_cpu = 0
    count_cpu_inf = 0
    count_ram_sup = 0
    count_ram = 0
    count_ram_inf = 0

    for i in range(len(df)):
        cpu = df.iloc[i]["number_cpu"]
        ram = min(ram_config, key=lambda x:abs(x-df.iloc[i]["ram_server"]/1000))
        new_cpu = df.iloc[i]["new_cpu_optim"]
        new_ram = df.iloc[i]["new_ram_optim"]
        item_type = df.iloc[i]["item_type"]
        if item_type == "cpu":
            if new_cpu > cpu:
                count_cpu_sup += 1
            elif new_cpu == cpu:
                count_cpu += 1
            else:
                count_cpu_inf += 1
        elif item_type == "mem":
            if new_ram > ram:
                count_ram_sup += 1
            elif new_ram == ram:
                count_ram += 1
            else:
                count_ram_inf += 1

    print( count_cpu_sup, count_cpu, count_cpu_inf, count_ram_sup, count_ram, count_ram_inf )
    return df_item_ts_30
                
df_item_ts_30 = config_optim_season()
df_item_ts_30.head()

,ds,yhat,yhat_upper,itemid,yhat_lower,item_type,name_server,number_cpu,ram_server,new_cpu_optim,new_ram_optim
335,2022-11-21,83.961987,94.711608,606770,71.351254,mem,SWPAFRETRADM2,1,4,1,8
336,2022-11-22,85.338458,97.186682,606770,73.753842,mem,SWPAFRETRADM2,1,4,1,8
337,2022-11-23,86.187269,98.328944,606770,74.317176,mem,SWPAFRETRADM2,1,4,1,8
338,2022-11-24,85.759144,96.942469,606770,73.265879,mem,SWPAFRETRADM2,1,4,1,8
339,2022-11-25,85.840877,97.490782,606770,73.600827,mem,SWPAFRETRADM2,1,4,1,8
340,2022-11-26,83.335654,94.342250,606770,71.400637,mem,SWPAFRETRADM2,1,4,1,8
341,2022-11-27,66.776920,78.231063,606770,54.612593,mem,SWPAFRETRADM2,1,4,1,4
342,2022-11-28,83.806738,96.198849,606770,72.128521,mem,SWPAFRETRADM2,1,4,1,8
343,2022-11-29,85.183055,97.159464,606770,73.056271,mem,SWPAFRETRADM2,1,4,1,8
344,2022-11-30,86.031711,98.177156,606770,73.010203,mem,SWPAFRETRADM2,1,4,1,8


In [13]:

def config_constraint_prio_season(df=df_item_ts_30, threshold_min_cpu=60, threshold_max_cpu=80, threshold_min_ram=70, threshold_max_ram=80, max_cpu=35, max_ram=0):    
    
    cpu_config = [1,2,4,8,16]
    ram_config = [2,4,6,8,12,16,24,32,64,128]
    lnew_cpu=[]
    lnew_ram=[]
    max_cpu = max_cpu * 30
    max_ram = max_ram * 30

    for i in range(len(df)):
        item_type = df.iloc[i]["item_type"]
        usage_max = df.iloc[i]["yhat_upper"]
        usage_min = df.iloc[i]["yhat_lower"]
        cpu = df.iloc[i]["number_cpu"]
        ram = df.iloc[i]["ram_server"]

        if item_type == 'cpu':
            lnew_ram.append(ram)
            if usage_max != 0:
                if usage_min < threshold_min_cpu and usage_max <= threshold_max_cpu:
                    ratio = round(threshold_min_cpu/usage_min)
                    new_cpu = min(cpu_config, key=lambda x:abs(x-cpu/ratio))
                    max_cpu = max_cpu + cpu - new_cpu
                    lnew_cpu.append(new_cpu)
                else:
                    lnew_cpu.append(cpu)
            else:
                lnew_cpu.append(cpu)

        elif item_type == 'mem':
            lnew_cpu.append(cpu)
            if usage_max != 0:
                if usage_min < threshold_min_ram and usage_max <= threshold_max_ram:
                    ratio = round(threshold_min_ram/usage_min)
                    new_ram = min(ram_config, key=lambda x:abs(x-ram/ratio))
                    max_ram = max_ram + ram - new_ram
                    lnew_ram.append(new_ram)
                else:
                    lnew_ram.append(ram)
            else:
                lnew_ram.append(ram)
        else:
            lnew_cpu.append(cpu)
            lnew_ram.append(ram)

    df["new_cpu_constraint"] = lnew_cpu
    df["new_ram_constraint"] = lnew_ram

    df = df.sort_values("yhat_upper", ascending=False)
    for i in range(len(df)):

        item_type = df.iloc[i]["item_type"]
        usage_max = df.iloc[i]["yhat_upper"]
        usage_min = df.iloc[i]["yhat_lower"]
        cpu = df.iloc[i]["number_cpu"]
        ram = df.iloc[i]["ram_server"]

        if item_type == 'cpu':
            if usage_max > threshold_max_cpu:
                ratio = math.ceil(usage_max/threshold_max_cpu)
                new_cpu = min(cpu_config, key=lambda x:abs(x-cpu*ratio))
                if new_cpu <= max_cpu:
                    df.at[i, 'new_cpu'] = new_cpu
                    max_cpu = max_cpu + cpu - new_cpu

        elif item_type == 'mem':
            if usage_max > threshold_max_ram:
                ratio = math.ceil(usage_max/threshold_max_ram)
                new_ram = min(ram_config, key=lambda x:abs(x-ram*ratio))
                if new_ram <= max_ram:
                    df.at[i, 'new_ram'] = new_ram
                    max_ram = max_ram + ram - new_ram
    
    count_cpu_sup = 0
    count_cpu = 0
    count_cpu_inf = 0
    count_ram_sup = 0
    count_ram = 0
    count_ram_inf = 0


    for i in range(len(df)):
        cpu = df.iloc[i]["number_cpu"]
        ram = min(ram_config, key=lambda x:abs(x-df.iloc[i]["ram_server"]/1000))
        new_cpu = df.iloc[i]["new_cpu_constraint"]
        new_ram = df.iloc[i]["new_ram_constraint"]
        item_type = df.iloc[i]["item_type"]
        if item_type == "cpu":
            if new_cpu > cpu:
                count_cpu_sup += 1
            elif new_cpu == cpu:
                count_cpu += 1
            else:
                count_cpu_inf += 1
        elif item_type == "mem":
            if new_ram > ram:
                count_ram_sup += 1
            elif new_ram == ram:
                count_ram += 1
            else:
                count_ram_inf += 1

    print( count_cpu_sup, count_cpu, count_cpu_inf, count_ram_sup, count_ram, count_ram_inf )

    max_cpu = round(max_cpu/30)
    max_ram = round(max_ram/30)
    print ("remaining number of cpu: " + str(max_cpu) + ", remaining number of ram: " + str(max_ram)) 
    return df_item_ts_30

df_item_ts_30 = config_constraint_prio_season()
df_item_ts_30.head()

0 5219 4501 6502 608 0
remaining number of cpu: 91, remaining number of ram: 0


,ds,yhat,yhat_upper,itemid,yhat_lower,item_type,name_server,number_cpu,ram_server,new_cpu_constraint,new_ram_constraint
335,2022-11-21,83.961987,94.711608,606770,71.351254,mem,SWPAFRETRADM2,1,4,1,4
336,2022-11-22,85.338458,97.186682,606770,73.753842,mem,SWPAFRETRADM2,1,4,1,4
337,2022-11-23,86.187269,98.328944,606770,74.317176,mem,SWPAFRETRADM2,1,4,1,4
338,2022-11-24,85.759144,96.942469,606770,73.265879,mem,SWPAFRETRADM2,1,4,1,4
339,2022-11-25,85.840877,97.490782,606770,73.600827,mem,SWPAFRETRADM2,1,4,1,4


## EXPORT OUTPUTS

In [ ]:
#df_appli_usage.head(50)
#df_item_ts_30.head(50)

In [14]:
df_appli_usage.to_csv(r'./Data/extraction/global_scenarios.csv', index=False)
df_item_ts_30.to_csv(r'./Data/extraction/periodic_scenarios.csv', index=False)
